In [ ]:
%pip install -U -q google-genai gradio

# Sumarización con Google Gemini API y Gradio
Esta interfaz te permite sumarizar textos utilizando el modelo Gemini de Google.

In [ ]:
from google.colab import userdata
import os

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google import genai
from google.genai import types
import gradio as gr

cliente = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
MODEL_ID = "gemini-2.0-flash-lite" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-05-20","gemini-2.5-pro-preview-05-06"] {"allow-input":true, isTemplate: true}

# Texto de ejemplo

In [ ]:
# Texto de ejemplo que se cargará por defecto
texto_ejemplo = """Estimado Amazon, la semana pasada pedí una figura de acción de Optimus Prime
en su tienda en línea en Alemania. Desafortunadamente, cuando abrí el paquete,
descubrí con horror que me habían enviado una figura de acción de Megatron
en su lugar. Como enemigo de toda la vida de los Decepticons, espero que pueda
entender mi dilema. Para resolver el problema, exijo un cambio de Megatron por
la figura de Optimus Prime que pedí. Adjunto copias de mis registros relativos
a esta compra. Espero tener noticias suyas pronto. Atentamente, Bumblebee."""

## Función de Sumarización

In [ ]:
def sumarizar_texto(texto, tipo_resumen):
    """Función que sumariza el texto usando Gemini"""

    if not texto.strip():
        return "Por favor, ingresa un texto para sumarizar."

    # Definir diferentes tipos de prompt según la opción seleccionada
    prompts = {
        "Rápido (2 oraciones)": f"Sumariza el siguiente texto en dos oraciones de rápida lectura:\n\nTexto: {texto}",
        "Detallado (3-4 líneas)": f"Resumí el siguiente texto en no más de 4 líneas:\n\nTexto: {texto}",
        "Muy breve (1 oración)": f"Resumí el siguiente texto en UNA sola oración:\n\nTexto: {texto}"
    }

    try:
        respuesta = cliente.models.generate_content(
            model=MODEL_ID,
            contents=[prompts[tipo_resumen]]
        )
        return respuesta.text

    except Exception as e:
        return f"Error al procesar el texto: {str(e)}"

## Interface Gradio

In [ ]:
# Crear la interfaz Gradio
interface = gr.Interface(
    fn=sumarizar_texto,
    inputs=[
        gr.Textbox(
            label="Texto a sumarizar",
            placeholder="Pega acá el texto que queres sumarizar...",
            lines=8,
            value=texto_ejemplo
        ),
        gr.Dropdown(
            choices=["Rápido (2 oraciones)", "Detallado (3-4 líneas)", "Muy breve (1 oración)"],
            label="Tipo de resumen",
            value="Rápido (2 oraciones)"
        )
    ],
    outputs=gr.Textbox(
        label="Resumen generado",
        lines=4
    ),
    title="Sumarizador con Gemini",
    description="Ingresa cualquier texto y elegí el tipo de resumen que preferis. El modelo Gemini generará un resumen automáticamente.",
    examples=[
        ["Estimado Amazon, la semana pasada pedí una figura de acción de Optimus Prime en su tienda en línea en Alemania.", "Rápido (2 oraciones)"],
    ],
    theme=gr.themes.Soft()
)

# Lanzar la interfaz
interface.launch()